In [1]:
from pyspark.sql import SparkSession

In [71]:
spark = (
    SparkSession
    .builder
    .appName("home-load-tables-iceberg")
    .master("spark://f7f8bb82a877:7077")
    .config("spark.executor.cores", 1)
    .config("spark.cores.max", 6)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

24/12/31 17:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [72]:
spark

In [7]:
%%sql

SHOW TABLES IN SolarX_Raw_Transactions

24/12/31 14:30:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace,tableName,isTemporary
SolarX_Raw_Transactions,home_power_readings,False
SolarX_Raw_Transactions,solar_panel,False
SolarX_Raw_Transactions,solar_panel_readings,False


# Create a new name-space/database for the warehouse

In [ ]:
%%sql

CREATE DATABASE IF NOT EXISTS SolarX_WH

# Home Power Usage WH

## Investigate home raw date

In [8]:
%%sql

DESCRIBE SolarX_Raw_Transactions.home_power_readings

col_name,data_type,comment
timestamp,timestamp,None
15_minutes_interval,int,None
min_consumption_wh,float,None
max_consumption_wh,float,None
,,
# Partitioning,,
Part 0,days(timestamp),
Part 1,15_minutes_interval,


In [9]:
%%sql

SELECT * FROM SolarX_Raw_Transactions.home_power_readings LIMIT 5

timestamp,15_minutes_interval,min_consumption_wh,max_consumption_wh
2013-01-01 03:30:00,10,0.0010607765289023519,0.006515521090477705
2013-01-01 03:30:00.005000,10,0.0010607768781483173,0.00651552015915513
2013-01-01 03:30:00.010000,10,0.0010607772273942828,0.006515519227832556
2013-01-01 03:30:00.015000,10,0.0010607774602249265,0.006515517830848694
2013-01-01 03:30:00.020000,10,0.001060777809470892,0.006515516899526119


In [50]:
%%sql

SELECT 
    timestamp,
    DATE(timestamp) as date,
    15_minutes_interval,
    min_consumption_wh,
    max_consumption_wh
FROM 
    SolarX_Raw_Transactions.home_power_readings 
LIMIT 5

timestamp,date,15_minutes_interval,min_consumption_wh,max_consumption_wh
2013-01-03 22:15:00,2013-01-03,85,0.0009668199927546084,0.005331419408321381
2013-01-03 22:15:00.005000,2013-01-03,85,0.0009668199345469475,0.005331420339643955
2013-01-03 22:15:00.010000,2013-01-03,85,0.0009668199345469475,0.005331421736627817
2013-01-03 22:15:00.015000,2013-01-03,85,0.0009668198763392866,0.005331423133611679
2013-01-03 22:15:00.020000,2013-01-03,85,0.0009668198181316257,0.005331424530595541


### Query low frequency data for wh

In [39]:
%%sql

SELECT
    CONCAT(DATE(timestamp), '--', 15_minutes_interval) as home_power_readings_id,
    DATE(timestamp) as date,
    15_minutes_interval,
    SUM(min_consumption_wh) as min_consumption_power_wh,
    SUM(max_consumption_wh) as max_consumption_power_wh
FROM 
    SolarX_Raw_Transactions.home_power_readings
WHERE 
    DAY(timestamp) = 1
GROUP BY 
    DATE(timestamp), 15_minutes_interval
SORT BY
    15_minutes_interval
LIMIT 10

home_power_readings_id,date,15_minutes_interval,min_consumption_power_wh,max_consumption_power_wh
2013-01-01--0,2013-01-01,0,210.4949002956273,1234.37793413084
2013-01-01--1,2013-01-01,1,210.90460421564057,1195.791982460767
2013-01-01--2,2013-01-01,2,211.31430813518818,1157.2060307911597
2013-01-01--3,2013-01-01,3,211.72401205578353,1118.620079122018
2013-01-01--4,2013-01-01,4,206.7439376907423,1116.9222173416056
2013-01-01--5,2013-01-01,5,196.37402515218128,1152.1128553207964
2013-01-01--6,2013-01-01,6,186.00411261827685,1187.3034932990558
2013-01-01--7,2013-01-01,7,175.6342000772711,1222.494131277781
2013-01-01--8,2013-01-01,8,175.57182770763757,1223.2657041028142
2013-01-01--9,2013-01-01,9,185.8171100311447,1189.6178293367848


## Home readings dimension and fact tables

### Home dimension

In [55]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_home PURGE

++
||
++
++

In [56]:
%%sql

CREATE TABLE SolarX_WH.dim_home(
    home_key                            INT         NOT NULL,
    home_id                             INT         NOT NULL,
    appliance                           VARCHAR(25) NOT NULL,    
    consumption_power                   FLOAT       NOT NULL,
    usage_time                          VARCHAR(50) NOT NULL,

    -- scd type2 for consumption_power
    consumption_power_start_date        TIMESTAMP   NOT NULL,
    consumption_power_end_date          TIMESTAMP,

    -- scd type2 for usage_time
    usage_time_start_date               TIMESTAMP  NOT NULL,
    usage_time_end_date                 TIMESTAMP
)
USING iceberg;

++
||
++
++

### Home fact

In [73]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.fact_home_power_readings PURGE

24/12/31 17:41:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [74]:
%%sql

CREATE TABLE SolarX_WH.fact_home_power_readings(
    home_power_reading_key          INT           NOT NULL,
    home_key                        SMALLINT      NOT NULL,   -- REFERENCES dim_home(home_key)
    date_key                        SMALLINT      NOT NULL,   -- REFERENCES dim_date(date_key)

    home_power_reading_id          VARCHAR(25)   NOT NULL,
    date                            DATE          NOT NULL,
    15_minutes_interval             SMALLINT      NOT NULL,
    min_consumption_power_wh        FLOAT         NOT NULL,
    max_consumption_power_wh        FLOAT         NOT NULL 
)

USING iceberg
PARTITIONED BY (MONTH(date), 15_minutes_interval)

++
||
++
++

## Solar panel readings dimension and fact tables

## Investigate solar panel raw date

In [52]:
%%sql

DESCRIBE SolarX_Raw_Transactions.solar_panel

col_name,data_type,comment
id,int,None
name,string,None
capacity_kwh,float,None
intensity_power_rating,float,None
temperature_power_rating,float,None


In [53]:
%%sql

DESCRIBE SolarX_Raw_Transactions.solar_panel_readings

col_name,data_type,comment
timestamp,timestamp,None
15_minutes_interval,int,None
panel_id,int,None
generation_power_wh,float,None
,,
# Partitioning,,
Part 0,days(timestamp),
Part 1,panel_id,
Part 2,15_minutes_interval,


### Query low frequency data for wh

In [61]:
%%sql

SELECT
    CONCAT(DATE(timestamp), '--', 15_minutes_interval) as solar_panel_power_readings_id,
    DATE(timestamp) as date,
    15_minutes_interval,
    SUM(generation_power_wh) as generation_power_wh
FROM 
    SolarX_Raw_Transactions.solar_panel_readings
WHERE 
    DAY(timestamp) = 1 AND panel_id = 1
GROUP BY 
    DATE(timestamp), 15_minutes_interval
SORT BY
    generation_power_wh desc
LIMIT 10

solar_panel_power_readings_id,date,15_minutes_interval,generation_power_wh
2013-01-01--34,2013-01-01,34,1112.372815785464
2013-01-01--32,2013-01-01,32,1108.504030722659
2013-01-01--37,2013-01-01,37,1075.1697589475662
2013-01-01--33,2013-01-01,33,1070.1509305494837
2013-01-01--39,2013-01-01,39,1033.8308916147798
2013-01-01--35,2013-01-01,35,1033.8161300485954
2013-01-01--38,2013-01-01,38,1001.8857660298236
2013-01-01--36,2013-01-01,36,997.2686491240747
2013-01-01--42,2013-01-01,42,929.7826389158145
2013-01-01--41,2013-01-01,41,923.0105679007247


### Solar panel dimension

In [54]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_solar_panel PURGE

++
||
++
++

In [57]:
%%sql

CREATE TABLE SolarX_WH.dim_solar_panel(
    solar_panel_key                             SMALLINT    NOT NULL,
    solar_panel_id                              SMALLINT    NOT NULL,
    name                                        VARCHAR(20) NOT NULL,    
    capacity_kwh                                FLOAT       NOT NULL,
    intensity_power_rating_wh                   FLOAT       NOT NULL,
    temperature_power_rating_c                  FLOAT       NOT NULL,

    -- scd type2 for capacity_kwh
    capacity_kwh_start_date                     TIMESTAMP   NOT NULL,
    capacity_kwh_end_date                       TIMESTAMP,

    -- scd type2 for intensity_power_rating_wh
    intensity_power_rating_wh_start_date        TIMESTAMP  NOT NULL,
    intensity_power_rating_wh_end_date          TIMESTAMP,

    -- scd type2 for temperature_power_rating_c
    temperature_power_rating_c_start_date       TIMESTAMP  NOT NULL,
    temperature_power_rating_c_end_date         TIMESTAMP
)
USING iceberg;

++
||
++
++

### Solar panel fact

In [66]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.fact_solar_panel_power_readings PURGE

++
||
++
++

In [67]:
%%sql

CREATE TABLE SolarX_WH.fact_solar_panel_power_readings(
    solar_panel_reading_key         INT           NOT NULL,
    solar_panel_key                 SMALLINT      NOT NULL,   -- REFERENCES dim_solar_panel(solar_panel_key)
    date_key                        SMALLINT      NOT NULL,   -- REFERENCES dim_date(date_key)

    solar_panel_reading_id          VARCHAR(25)   NOT NULL,
    date                            DATE          NOT NULL,
    15_minutes_interval             SMALLINT      NOT NULL,
    generation_power_wh             FLOAT         NOT NULL 
)

USING iceberg
PARTITIONED BY (MONTH(date), 15_minutes_interval)

++
||
++
++

In [70]:
spark.stop()